# Entropy, Compression, and Huffman Variable Length Encoding
stough 202-

Images and videos can be quite large. At 60 frames per second, a 4K Ultra HD movie of 2 hours could require 

\begin{equation*}
((3840\cdot2160)pixels \cdot 3\frac{byte}{pixel} = 24883200 \frac{byte}{frame}\\
\frac{24883200 \frac{byte}{frame}\cdot60\frac{frame}{second}\cdot60\frac{second}{minute}\cdot120 minutes}{10^9\frac{byte}{GB}} = 10749.5 GB,
\end{equation*}

or about 163 blu-ray discs to store without compression. So clearly compression is a thing. 

Compression takes advantage of different kinds of redundancy in the signal. Images in particular have a lot of redundancy. Spatial and Temporal redundancy are characterized by the fact that most pixels are closely related their neighbors. This is due to the fact that any images of human interest are likely to have objects comprised of regions of similar color. *Temporal* redundancy is the video case, where a pixel over time changes little usually. 

Coding redundancy is the one we're dealing with here. Most images we consider are 3-channel images with 8 bits per pixel per channel. We need 8 bits to represent a value in $[0, 255]$. Variable-length encoding can take advantage of the differential frequency of certain pixel values over others, encoding more common pixels with fewer bits, and paying that off by using more bits to represent less common pixels.

- **Compressibility**: The degree to which the information in an image or video can be represented using fewer bits. In the example above of the highly compressible 4K Ultra HD, we might say the *compression ratio* is ~160 (display size over storage size). 
- **Entropy**: a measure of the information content in a signal. 
    - signal: here, think of this as seeing each individual pixel in the image in order.
    - Think of entropy as the surprise, or "unpredictability" of the signal. Each new pixel value $k$ in the signal gives you some information,
    \begin{equation*}
    I(k) = -\log_2(p(k)),
    \end{equation*}
    where $p(k)$ is the probability of observing $k$ in the signal. For example, if the probability of observing $k$ is 1, as in 100\% of all the pixels have the value $k$, then there is no information/surpise, or 0 bits, associated with observing it. If $p(k) = \frac{1}{2}$, then that's 1 bit of information. (We won't consider the symbols $\kappa$ that *never* show up, since observing such a symbol would be infinitely surprising :-). 
    - For our images in the range $[0, 255]$, we can measure the entropy or information content of the image as 
    \begin{equation*}
    \tilde{H} = -\sum_{k=0}^{255} p(k)\log_{2}p(k)
    \end{equation*}
    This is just an average of how many bits of information each newly observed pixel gives us, weighted by how likely that pixel value is. $p(k)$ is just the normalized histogram values.
    
In this notebook we'll explore Huffman variable-length encoding its effectiveness in some images.

## Imports
Notice I've added our own custom Huffman utilities `huff_utils` and [`scipy.stats.entropy`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.entropy.html). Study `build_huff_tree` in particular.

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import skimage.color as color
from ipywidgets import VBox, HBox, FloatLogSlider
from scipy.stats import entropy

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')


from huff_utils import (build_huff_tree,
                        build_huff_encoder,
                        build_huff_pair,
                        load_huffable_image,
                        test_tree_making)
from matrix_utils import (arr_info,
                          make_linmap)
from vis_utils import (vis_rgb_cube,
                       vis_hsv_cube,
                       vis_hists,
                       vis_pair,
                       lab_uniform)

## A06 Test encoding/decoding.

In [2]:
def decodeImage(enI, decoder):
    deI=[]
    keys = decoder.keys()
    i=1
    while len(enI) >= i:
        if enI[:i] in keys:
            deI.append(decoder[enI[0:i]])
            enI = enI[i:]
            i=1
        else:
            i+=1
    return deI

In [3]:
I = plt.imread('../dip_pics/yes_small.jpg')
plt.figure(figsize=(4,4))
plt.imshow(I)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
def huff_compress(I):
    reI = np.zeros(I.shape)
    for channel in range(3):
        Ic = load_huffable_image(I[...,channel].copy())
        encoder, decoder = build_huff_pair(Ic)
        enI = ''.join(encoder[pix] for pix in Ic.ravel())
        deI = decodeImage(enI, decoder)
        reI[...,channel]=np.reshape(deI, Ic.shape)
    return reI

In [5]:
reI = huff_compress(I)

In [6]:
f, axarr = plt.subplots(1,2,figsize=(8,8))
axarr[0].imshow(I.astype('uint8'))
axarr[1].imshow(reI.astype('uint8'))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
def statR(I):
    #red
    Ih = load_huffable_image(I[..., 0].copy())
    encoder, decoder = build_huff_pair(Ih)
    print("Channel Red statistics:")
    
    #8-bit size
    size_8bit = np.prod(Ih.shape)/1024
    
    #Huffman size
    enIh = ''.join([encoder[pix] for pix in Ih.ravel()])
    size_huff = len(enIh)/8/1024
    
    bins = np.arange(257)
    freq, bins = np.histogram(Ih.ravel(), bins)
    print("Red channel entropy is %.5f bits" % entropy(freq, base=2))
    print("Size at 8-bit encoding: %.2fKB." % size_8bit)
    print("Size with huff encoding: %.2fKB, or %.2f bits per pixel" % (size_huff, len(enIh)/np.prod(Ih.shape)))
    print("---------------")

In [8]:
def statG(I):
    #green
    Ih = load_huffable_image(I[..., 1].copy())
    encoder, decoder = build_huff_pair(Ih)
    print("Channel Grn statistics:")
    
    #8-bit size
    size_8bit = np.prod(Ih.shape)/1024
    
    #Huffman size
    enIh = ''.join([encoder[pix] for pix in Ih.ravel()])
    size_huff = len(enIh)/8/1024
    
    bins = np.arange(257)
    freq, bins = np.histogram(Ih.ravel(), bins)
    print("Grn channel entropy is %.5f bits" % entropy(freq, base=2))
    print("Size at 8-bit encoding: %.2fKB." % size_8bit)
    print("Size with huff encoding: %.2fKB, or %.2f bits per pixel" % (size_huff, len(enIh)/np.prod(Ih.shape)))
    print("---------------")

In [12]:
def statB(I):
    #blue
    Ih = load_huffable_image(I[..., 2].copy())
    encoder, decoder = build_huff_pair(Ih)
    print("Channel Blu statistics:")
    
    #8-bit size
    size_8bit = np.prod(Ih.shape)/1024
    
    #Huffman size
    enIh = ''.join([encoder[pix] for pix in Ih.ravel()])
    size_huff = len(enIh)/8/1024
    
    bins = np.arange(257)
    freq, bins = np.histogram(Ih.ravel(), bins)
    print("Blu channel entropy is %.5f bits" % entropy(freq, base=2))
    print("Size at 8-bit encoding: %.2fKB." % size_8bit)
    print("Size with huff encoding: %.2fKB, or %.2f bits per pixel" % (size_huff, len(enIh)/np.prod(Ih.shape)))
    print("---------------")

In [13]:
def getCompressionStats(filename):
    I = plt.imread(filename)
    statR(I)
    statG(I)
    statB(I)

In [16]:
getCompressionStats('../dip_pics/yes_small.jpg')

Channel Red statistics:
Red channel entropy is 5.94605 bits
Size at 8-bit encoding: 39.85KB.
Size with huff encoding: 29.80KB, or 5.98 bits per pixel
---------------
Channel Grn statistics:
Grn channel entropy is 5.79043 bits
Size at 8-bit encoding: 39.85KB.
Size with huff encoding: 29.02KB, or 5.83 bits per pixel
---------------
Channel Blu statistics:
Blu channel entropy is 6.26437 bits
Size at 8-bit encoding: 39.85KB.
Size with huff encoding: 31.36KB, or 6.30 bits per pixel
---------------
